In [1]:
import torch
from diffusers import ZImagePipeline
import polars as pl
import os
from tqdm import tqdm

D:\conda\envs\txt2img\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "D:\conda\envs\txt2img\lib\site-packages\xformers\__init__.py", line 57, in _is_triton_available
    import triton  # noqa
ModuleNotFoundError: No module named 'triton'


In [2]:
torch.set_grad_enabled(False)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [3]:
out_dir = "../../imgs"
os.makedirs(out_dir, exist_ok=True)

In [4]:
pipe = ZImagePipeline.from_pretrained(
    "D:\GraduateProject\model\Tongyi-MAI\Z-Image-Turbo",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)
pipe.to("cuda")

pipe.safety_checker = None
pipe.feature_extractor = None

Loading pipeline components...:  60%|██████    | 3/5 [00:09<00:06,  3.44s/it]`torch_dtype` is deprecated! Use `dtype` instead!

Loading pipeline components...: 100%|██████████| 5/5 [00:09<00:00,  1.95s/it]


In [5]:
news = pl.read_parquet('../../Data/news.parquet')

In [6]:
def build_news_prompt_with_text(row):
    title = row["title"]
    category = row["category"]
    abstract = row["abstract"] or "以该新闻主题为灵感，抽象地表现情绪和场景"
    words = abstract.split()

    return f"""
        生成{category}新闻封面图像，要求如下：
        1. 仅在封面中显示新闻标题文本：“{title}”，字体清晰突出。
        2. 新闻内容或摘要：{abstract}，请通过图像、场景、人物表达主题。
        3. 图片保证清晰可见。
        """

In [ ]:
for row in news.iter_rows(named=True):
    nid=row['news_id']
    out_path = f"{out_dir}/{nid}.png"

    if os.path.exists(out_path):
        continue

    try:
        with torch.inference_mode():
            prompt = build_news_prompt_with_text(row)
            image = pipe(
                prompt=prompt,
                height=512,
                width=512,
                num_inference_steps=30,
                guidance_scale=0.0,
            ).images[0]
        image.save(out_path)
    except Exception as e:
        print(f"[ERROR] {nid}: {e}")
        torch.cuda.empty_cache()


 90%|█████████ | 27/30 [03:47<00:23,  7.67s/it]